# Visualization of graph filters

[Nathanaël Perraudin](http://perraudin.info), [Michaël Defferrard](http://deff.ch), Tomasz Kacprzak

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import pygsp
import healpy as hp
from scnn import utils
import matplotlib.pyplot as plt
import numpy as np


In [ ]:
plt.rcParams['figure.figsize'] = (10, 10)

Create the graph

In [ ]:
Nside = 64
G = utils.healpix_graph(nside=Nside)

Create some dummy filters

In [ ]:
tau = [1,5,10,20,50,200,1000]
hf = pygsp.filters.Heat(G, tau=tau)
fig, ax = plt.subplots()
hf.plot(eigenvalues=False, sum=False, ax=ax)
_ = ax.set_title('Filter frequency response')

In [ ]:
order = 30
ind0 = 0
def localize_filter(hf, ind0=0, order=10):
    sig = np.zeros([hf.G.N])
    sig[ind0]=1
    conv = hf.analyze(sig,order=order)
    return conv
conv = localize_filter(hf, ind0, order)

def plot_filters(conv, ind0=0, order=10):
    Nside = hp.npix2nside(conv.shape[0])
    nfilt = conv.shape[1]
    reso = hp.pixelfunc.nside2resol(nside=Nside,arcmin=True)*order/70
    colatitude_deg, longitude_deg = hp.pix2ang(nside=Nside,ipix = ind0, nest=True, lonlat=True)
    rot=[colatitude_deg, longitude_deg]
    print(rot)
    sy = int(np.ceil(np.sqrt(nfilt)))
    sx = int(np.ceil(nfilt/sy))
    for i in range(nfilt):
        hp.gnomview(conv[:,i], nest=True, rot=rot, reso=reso, sub=(sx,sy,i+1), title='Filter {}'.format(i), notext=True)
plot_filters(conv, ind0=ind0, order=order)

In [ ]:
def plot_filers_section(hf, order=10):
    Nside = hp.npix2nside(hf.G.N)
    npix = hp.nside2npix(Nside)
    # Create an inverse mapping from nest to ring
    index = hp.reorder(list(range(npix)), n2r=True)
    # Select the point in the middle of the equator
    ind1 = index[npix//2]
    # Localize the filter
    conv = localize_filter(hf, ind0=ind1, order=order)
    # Get the value on the equator back
    val = order
    equator_part = list(range(npix//2-val,npix//2+val))
    nfilt = conv.shape[1] # number of filers
    # Make the x axis (angular position of the nodes in degree)
    rad = [hp.pix2ang(Nside,el,nest=False)[1] for el in equator_part]
    rad_middle = abs(rad[-1]+rad[0])/2
    x_axis = [(el-rad_middle)/np.pi/2*360 for el in rad]
    # Plot everything
    sy = int(np.ceil(np.sqrt(nfilt)))
    sx = int(np.ceil(nfilt/sy))
    fig, axes = plt.subplots(sx, sy)
    el = 0
    for i in range(sx):
        for j in range(sy):
            axes[i,j].plot(x_axis,conv[index[equator_part],el])
            axes[i,j].set_title('Filter {}'.format(el))
#             axes[i,j].set_xlabel('Degree')
            el = el+1
            if el==nfilt:
                break
plot_filers_section(hf,order)

We see the polynomial approximation error on the filter 6.